In [ ]:
import treelite
import tl2cgen
import xgboost as xgb
import numpy as np
import random
import matplotlib.pyplot as plt
import platform
import re
import pandas as pd
from scipy.stats import gmean
from sklearn.preprocessing import LabelEncoder
from ucimlrepo import fetch_ucirepo

In [ ]:
datasets_names = ['Breast Cancer', 'Rice', 'Wine', 'Magic', 'Glass']
nr_instances_validation = 20
tree_depths = [2, 3, 5, 8]
nr_validation_reuses = 5000
nr_root_nodes = 7

In [ ]:
# You don't have to run this for data analysis
datasets = [fetch_ucirepo(id=17),  fetch_ucirepo(id=545), fetch_ucirepo(id=109), fetch_ucirepo(id=159), fetch_ucirepo(id=42)]

In [ ]:
# Definately don't run this for just data analysis
def create_c_array_string(array):
    array_string = "{\n"
    array_string += ",\n".join("{" + ", ".join(str(x) for x in row) + "}" for row in array)
    array_string += "\n}"
    return array_string

def count_decimals(number):
    str_num = str(number)
    if '.' not in str_num:
        return 0
    return len(str_num) - str_num.index('.') - 1

def beg_c_file_str() :
    a = """
#include <stdio.h>
#include <inttypes.h>
#include "sdkconfig.h"
#include "freertos/FreeRTOS.h"
#include "freertos/task.h"
#include "esp_chip_info.h"
#include "esp_flash.h"
#include "esp_system.h"
#include "esp_timer.h"
#include <stdlib.h>
#include <string.h>
#include <float.h>
#include <math.h>
#include <stdint.h>

union Entry {
  int missing;
  float fvalue;
  int qvalue;
};

double elapsed_time;
int64_t start, end;


"""
    return a

def end_c_file_str() :
    a= """
    for (int i = 10; i >= 0; i--) {
        printf("Restarting in %d seconds...\\n", i);
        vTaskDelay(1000 / portTICK_PERIOD_MS);
    }
    printf("Restarting now.\\n");
    fflush(stdout);
    esp_restart();
    """
    return a


def makeTimingString(val_nr, nr_results, reuses, treedepth, rootnodes):
    a = """
//////////////////////////////////////////////

start = esp_timer_get_time();
for (int z = 0; z < """+str(reuses)+"""; z++) {
    for (int j = 0; j < sizeof(validation_instances"""+str(val_nr)+""")/sizeof(validation_instances"""+str(val_nr)+"""[0]); j++) {
        float result"""+str(val_nr)+"""["""+str(nr_results)+"""]={0.0f};
        predictN"""+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"""(validation_instances"""+str(val_nr)+"""[j], 0, &result"""+str(val_nr)+""");
    }
}
end = esp_timer_get_time();
elapsed_time = ((double) (end - start)) / 1000000.0;
printf("Time elapsed: %f, for N"""+str(val_nr)+'m'+str(treedepth)+'td'+str(rootnodes)+'rn'+""" \\n", elapsed_time);

start = esp_timer_get_time();
for (int z = 0; z < """+str(reuses)+"""; z++) {
    for (int j = 0; j < sizeof(validation_instances"""+str(val_nr)+""")/sizeof(validation_instances"""+str(val_nr)+"""[0]); j++) {
        float result"""+str(val_nr)+"""["""+str(nr_results)+"""]={0.0f};
        predictQ"""+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"""(validation_instances"""+str(val_nr)+"""[j], 0, &result"""+str(val_nr)+""");
    }
}
end = esp_timer_get_time();
elapsed_time = ((double) (end - start)) / 1000000.0;
printf("Time elapsed: %f, for Q"""+str(val_nr)+'m'+str(treedepth)+'td'+str(rootnodes)+'rn'+""" \\n", elapsed_time);


start = esp_timer_get_time();
for (int z = 0; z < """+str(reuses)+"""; z++) {
    for (int j = 0; j < sizeof(validation_instances"""+str(val_nr)+""")/sizeof(validation_instances"""+str(val_nr)+"""[0]); j++) {
        float result"""+str(val_nr)+"""["""+str(nr_results)+"""]={0.0f};
        predictF"""+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"""(validation_instances"""+str(val_nr)+"""[j], 0, &result"""+str(val_nr)+""");
    }
}

end = esp_timer_get_time();
elapsed_time = ((double) (end - start)) / 1000000.0;
printf("Time elapsed: %f, for F"""+str(val_nr)+'m'+str(treedepth)+'td'+str(rootnodes)+'rn'+""" \\n", elapsed_time);


//////////////////////////////////////////////
"""
    return a

def makeFunctionString(val_nr, treedepth, rootnodes, features):

    # Copy predict function from NO PARAM model to function string
    a="void predictN"+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"(union Entry* data, int pred_margin, float* result) {\n"
    with open("./Trees/model"+str(val_nr)+"N"+str(treedepth)+"td"+str(rootnodes)+"rn"+"/main.c", "r") as f:
        lines = f.readlines()

    start_line = next(i for i, line in enumerate(lines) if 'void predict(union Entry* data, int pred_margin, float* result) {' in line)
    end_line = next(i for i, line in enumerate(lines) if 'if (!pred_margin) { postprocess(result);' in line)
    content = lines[start_line+1:end_line]
    a = a + "".join(content) + "}\n\n"

    # Copy predict function from FLINT model to function string
    a+="void predictF"+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"(union Entry* data, int pred_margin, float* result) {\n"
    with open("./Trees/model"+str(val_nr)+"F"+str(treedepth)+"td"+str(rootnodes)+"rn"+"/main.c", "r") as f:
        lines = f.readlines()

    start_line = next(i for i, line in enumerate(lines) if 'void predict(union Entry* data, int pred_margin, float* result) {' in line)
    end_line = next(i for i, line in enumerate(lines) if 'if (!pred_margin) { postprocess(result);' in line)
    content = lines[start_line+1:end_line]
    a = a + "".join(content) + "}\n\n"



    # Copy predict function from QUANTIZE model to function string, including the quantization function itself
    a+="int quantize"+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"(float, unsigned);\n"

    a+="void predictQ"+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"(union Entry* data, int pred_margin, float* result) {\n"
    with open("./Trees/model"+str(val_nr)+"Q"+str(treedepth)+"td"+str(rootnodes)+"rn"+"/main.c", "r") as f:
        lines = f.readlines()

    start_line = next(i for i, line in enumerate(lines) if 'void predict(union Entry* data, int pred_margin, float* result) {' in line)
    end_line = next(i for i, line in enumerate(lines) if 'if (!pred_margin) { postprocess(result);' in line)
    lines[start_line+2] = "const unsigned char is_categorical[] = {"+''.join(['0, ' for i in range(features)]) + '};\n'
    lines[start_line+5] = "      data[i].qvalue = quantize"+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"(data[i].fvalue, i);"
    content = lines[start_line+1:end_line]

    
    a = a + "".join(content) + "}\n\nint quantize"+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"(float val, unsigned fid) {\n"
    with open("./Trees/model"+str(val_nr)+"Q"+str(treedepth)+"td"+str(rootnodes)+"rn"+"/quantize.c", "r") as f:
        lines = f.readlines()
    start_line = next(i for i, line in enumerate(lines) if 'int quantize(float val, unsigned fid) {' in line)
    a = a + "".join(lines[2:start_line-6])+"\n"
    content = lines[start_line+1:]
    a = a + "".join(content) + "\n"


    return a

c_file_str = beg_c_file_str()
timing_str = r''
functions_str = r''
nr_features = []
validation_instances = []
validation_instances_str = r''


for i, dataset in enumerate(datasets):
    for depth in tree_depths:
        # Train models
        X = dataset.data.features 
        y = dataset.data.targets 
        le = LabelEncoder()
        y = le.fit_transform(y)

        X.columns = X.columns.str.replace('[', '')
        X.columns = X.columns.str.replace(']', '')
        X.columns = X.columns.str.replace('<', '')
        dtrain = xgb.DMatrix(X, label=y)
        params = {"max_depth": depth, "eta": 0.1, "objective": "multi:softprob", "eval_metric": "mlogloss", "num_class": len(np.unique(y))}
        bst = xgb.train(params, dtrain, num_boost_round=nr_root_nodes, evals=[(dtrain, 'train')])
        model = treelite.Model.from_xgboost(bst)

        path_no_param = "./Trees/model"+str(i)+"N"+str(depth)+"td"+str(nr_root_nodes)+"rn"
        path_flint = "./Trees/model"+str(i)+"F"+str(depth)+"td"+str(nr_root_nodes)+"rn"
        path_quantize = "./Trees/model"+str(i)+"Q"+str(depth)+"td"+str(nr_root_nodes)+"rn"

        # Generate model C code
        tl2cgen.generate_c_code(model, dirpath=path_no_param, params={})
        tl2cgen.generate_c_code(model, dirpath=path_flint, params={"flint" : 1})
        tl2cgen.generate_c_code(model, dirpath=path_quantize, params={"quantize": 1})

        # Generate timing C code per predict function
        timing_str += makeTimingString(i, len(np.unique(y)), nr_validation_reuses, depth, nr_root_nodes)
        functions_str += makeFunctionString(i, depth, nr_root_nodes, X.shape[1])



    # Generate random validation instances with decimal precision equal to the maximum decimal precision of the training data
    min_values_feature = []
    max_values_feature = []
    nr_decimals = np.zeros((X.shape[0], X.shape[1]))
    max_decimals = []

    nr_results = len(np.unique(y))
    nr_features.append(X.shape[1])
    for k in range(min(X.shape[0],1000)):
        for j in range(X.shape[1]):
            nr_decimals[i][j] = count_decimals(X.iloc[k, j])

    for k in range(X.shape[1]):
        max_decimals.append(max(nr_decimals[:, k]))


    for k in range(X.shape[1]):
        min_values_feature.append(X.iloc[:, k].min())
        max_values_feature.append(X.iloc[:, k].max())

    validation_instances = np.zeros((nr_instances_validation, X.shape[1]))
    for j in range(nr_instances_validation):
        for k in range(X.shape[1]):
            validation_instances[j][k] = round(random.uniform(min_values_feature[k], max_values_feature[k]), int(max_decimals[k]))
    
    validation_instances_str += 'float validation_instances'+str(i)+'[][' + str(X.shape[1]) + '] = ' + create_c_array_string(validation_instances) + ';\n'


c_file_str += functions_str
c_file_str += "void app_main(void)\n{\n"
c_file_str += validation_instances_str
c_file_str += timing_str
c_file_str += end_c_file_str()
c_file_str += "\n}"
with open('generated_speedTestS3.c', 'w') as f:
    f.write(c_file_str)



In [ ]:
if 'times' in globals():
    original_times = times.copy()
else:
    original_times = [-1]
# Read out esp32times.txt
times = []
with open('esp32times.txt', 'r') as f:
    for line in f:
        match = re.search(r'\d+\.\d+', line)
        if match:
            times.append(float(match.group()))

if not times:
    print("The times are empty.")
else:
    times = np.array(times)
    for i in range(1, len(times), 3):
        times[i], times[i+1] = times[i+1], times[i]
    if np.array_equal(times, original_times):
        print("The times haven't been updated.")
    else:
        print(times)

In [ ]:
# Figure scaling so that 6 bar groups are 10 wide
plt.figure(figsize=((len(times)/3)*(10/6), 5))

x = np.arange(0, len(times), 3)

# Width of a bar 
width = 2

# Create the bar chart for each group with different colors
colors = ['red', 'green', 'blue']
labels = ['No parameters', 'Flint', 'Quantize']
for i in range(3):
    plt.bar(x + i*(width/3), times[i::3], color=colors[i], width=width/3, label=labels[i])

# Assuming 'names' is your array of group names
names = [f"{name}{depth}" for name in datasets_names for depth in tree_depths]

# Set the x-tick labels to the group names and remove the x-axis
plt.xticks(x-0.5 + width/2, names)
plt.tick_params(axis='x', length=0)

plt.ylabel('Seconds')
plt.title('Inference times')
plt.text(0, -0.1, 'Validation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nNumber of root nodes: '+str(nr_root_nodes)+'\nSystem info: ESP32-S3-DevKitC-1', ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()

In [ ]:
# Figure scaling so that 6 bar groups are 10 wide
plt.figure(figsize=((len(times)/(3*len(tree_depths)))*(10/6), 5))

speedups_flint = times[0::3]/times[1::3]
speedups_quantize = times[0::3]/times[2::3]

width = 2
speedups = np.ones(len(datasets_names)*3)
for i in range(len(datasets_names)):
    # Geometric mean for ratios
    speedups[i*3+1] = gmean(speedups_flint[i*len(tree_depths):i*len(tree_depths)+len(tree_depths)])
    speedups[i*3+2] = gmean(speedups_quantize[i*len(tree_depths):i*len(tree_depths)+len(tree_depths)])


x = np.arange(0, len(speedups), 3)

# Create the bar chart for each group with different colors
colors = ['red', 'green', 'blue']
labels = ['No parameters', 'Flint', 'Quantize']
for i in range(3):
    plt.bar(x + i*(width/3), speedups[i::3], color=colors[i], width=width/3, label=labels[i])


architecture = platform.architecture()

# Set the x-tick labels to the group names and remove the x-axis
plt.xticks(x-0.5 + width/2, datasets_names)
plt.tick_params(axis='x', length=0)

plt.ylabel('Speedup factor')
plt.title('Average speedup factors per dataset')
plt.text(0, -0.1, 'Validation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nNumber of root nodes: '+str(nr_root_nodes)+'\nSystem info: ESP32-S3-DevKitC-1', ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()

In [ ]:
times_no_parameters = [np.mean(times[i*3::len(tree_depths)*3]) for i in range(len(tree_depths))]
times_flint = [np.mean(times[i*3+1::len(tree_depths)*3]) for i in range(len(tree_depths))]
times_quantize = [np.mean(times[i*3+2::len(tree_depths)*3]) for i in range(len(tree_depths))]

architecture = platform.architecture()
# Create the line plot for each group with different colors
plt.plot(range(len(tree_depths)), times_no_parameters, color='red', marker='*', label='No parameters')
plt.plot(range(len(tree_depths)), times_flint, color='green', marker='^', label='Flint')
plt.plot(range(len(tree_depths)), times_quantize, color='blue', marker='s', label='Quantize')

# Set the x-tick labels to the values in tree_depths
plt.xticks(range(len(tree_depths)), tree_depths)

plt.xlabel('Maximum tree depth')
plt.ylabel('Time [s]')
plt.title('Average inference time per tree depth')

plt.text(0, -0.15, 'Datasets used: '+str(datasets_names)+'.\nValidation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nSystem info: ESP32-S3-DevKitC-1', ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()

In [ ]:
speedups_flint, speedups_quantize = [], []  
factors_flint = [times[i*3]/times[i*3+1] for i in range(len(datasets_names)*len(tree_depths))]
factors_quantize = [times[i*3]/times[i*3+2] for i in range(len(datasets_names)*len(tree_depths))]

for i in range(len(tree_depths)):
    # Geometric mean for ratios
    speedups_flint.append(gmean(factors_flint[i::len(tree_depths)]))
    speedups_quantize.append(gmean(factors_quantize[i::len(tree_depths)]))


architecture = platform.architecture()
# Create the line plot for each group with different colors
plt.plot(range(len(tree_depths)), np.ones(len(tree_depths)), color='red', marker='*', label='No parameters')
plt.plot(range(len(tree_depths)), speedups_flint, color='green', marker='^', label='Flint')
plt.plot(range(len(tree_depths)), speedups_quantize, color='blue', marker='s', label='Quantize')

# Set the x-tick labels to the values in tree_depths
plt.xticks(range(len(tree_depths)), tree_depths)

plt.xlabel('Maximum tree depth')
plt.ylabel('Speedup factor')
plt.title('Average speedup factor per tree depth')

plt.text(-0.1, -0.15, 'Datasets used:\n'+str(datasets_names)+'.\nValidation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nSystem info: ESP32-S3-DevKitC-1', ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()